# Databases

When we want to store data outside of our program, we might use a database to store this information. In this case we're using SQL (structured query language), in this case a database management system called SQLite that utilizes SQL to manipulate databases.

### Relational databases

A relational database allows us to store items that can be related to others in some way.

This data is usually structured in the form of tables, think excel spreadsheet with rows and columns.

**courses**

| row | course_number | course_name                          | unit_count |
|-----|---------------|--------------------------------------|------------|
| 1   | ICS 31        | Introduction to Programming          | 4          |
| 2   | ICS 32        | Programming with Software Libraries  | 4          |
| 3   | ICS 33        | Intermediate Programming with Python | 4          |

Each row of the table is some entry of data, and can essentially correspond to a tuple in Python, where the table defines the types stored in the tuple. It's almost as if the table above corresponds to the following namedtuple (although in this example I used NamedTuple from typing as opposed to namedtuple from collections):

In [1]:
from typing import NamedTuple

class Course(NamedTuple):
    row: int
    course_number: str
    course_name: str
    unit_count: int

# e.g.
ics_31 = Course(row = 1, course_number = 'ICS 31', course_name = 'Introduction to Programming', unit_count = 4)
ics_31

Course(row=1, course_number='ICS 31', course_name='Introduction to Programming', unit_count=4)

### Primary Keys

A _primary key_ is a subset of columns that are guaranteed to be unique for different rows. Consider storing a bunch of people in a database. We can't really use their names, because with a lot of people there's almost guaranteed to be two people who share identical names. This is why many places assign people IDs, like a student ID, or taxpayer ID to _uniquely_ identify you.

When we use a DBMS like sqlite, we can specify which columns represent the primary key and it will guarantee that:
1. No two rows have identical primary keys
2. We can associate a primary key to a row to find an entry quickly

The mechanism that a DBMS utilizes to guarantee this is quite similar to dictionaries, where are primary key would be the primary key of a row.

### Relationships

Since we can uniquely and quickly identify rows in a table with primary keys, it makes sense that we should be able to relate different elements to another through these primary keys.

Relationships have a _cardinality_, which is a limit on how many elements can have relationships with each other. Suppose we have tables $A, B$ (can sort of think of them as mathematical sets).

1. **one-to-one** - an element of $A$ can only be related to at most one element in $B$, and vise versa. e.g. (ignoring dual citizenship) one person can be associated to one passport, and vise versa

2. **one-to-many** - an element of $A$ can be related to many elements of $B$, but each element of $B$ can only be related to one element in $A$. e.g. a library member can have many books checked out a once, but any checked out book can only have been checked out by a single library member at once.

3. **many-to-many** - an element of $A$ can be related to many elements in $B$, and vise versa. e.g. I can "favorite" many songs on Spotify, and any song on Spotify can be favorited by many people. (although probably not the best example in terms of databases)

Many-to-many is the _most_ flexible, but when considering desgining relationships, we only want to be a flexible as we have to be.

### Foreign Keys

In order to actual implement these relationships, it makes sense to store what's called a _foreign key_, which is essentially the primary key of something you want to reference.

Usually with a DBMS we explicitly specify a column to store foreign keys, so it can enforce "referential integrity" which means what each row refers to should actually exist.

Since columns store scalar values, many-to-many relationships usually need to be implemented with some intermediary table that contains the foreign keys of related rows.

### SQL and SQLite

How do we apply this to actually managing a SQL db in python.

In [2]:
import sqlite3
connection = sqlite3.connect(':memory:')
type(connection)

sqlite3.Connection

This syntax allows us to establish a 'connection' with a database. In this case `':memory:'` denotes an in-memory database, but we could have instead passed in a filesystem path. This connection is our portal to execute SQL queries against our databases.

In [3]:
connection.execute(
    '''
    CREATE TABLE person(
        person_id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER
    ) STRICT;
    '''
)

Calling this `execute` method is how we pass SQL statements as a string for the sqlite to interpret and operate on our database. A few things to note about this specific query:
- **SQL keywords** or the tokens in caps are keywords that have some meaning in the SQL language
- Whitespaces and indentation don't matter which is why this query is valid despite being a weirdly formatted string
- Each statement is concluded with a semi colon

This syntax **creates** a **table** called **person**, which has the listed columns, with the types specified next to it, as well as `PRIMARY KEY` indicating that we want `person_id` to be a primary key.

Also the `STRICT` keyword means we will strictly enforce the types stored in these columns.

Also also, notice that this execute returned a `<sqlite3.Cursor>` object. If our statement had queried for data, it would be stored in said cursor, but creating a table doesn't so we don't really need to do anything with it.

In [4]:
cursor = connection.execute('SELECT name FROM sqlite_schema;')
print(cursor.fetchone())
print(cursor.fetchone())
cursor.close()

('person',)
None


SQLite retains certain metadata about how our db is structured, which we can also query for from the `sqlite_schema` table. What we've essentially done is `SELECT`ed all the rows, only asking for the `name` from each row, `FROM` the table `sqlite_schema`. Which only returned `('person',)`, since that's the only table we have.

The method `cursor.fetchone()` allows us to get these results one at a time, and it returns `NONE` once there are none left. Then we make sure to close the cursor when done.

### Insertion

Now that we've created a table, we presumably want to insert data to populated it with values.

In [5]:
connection.execute(
    '''
    INSERT INTO person (person_id, name, age)
    VALUES (1, 'Boo', 13);
    '''
)

`INSERT INTO x` indicates we want to insert into table `x`, and we intend to specify the columns in the following tuple `(person_id, name, age)`
Then the keyword `VALUES` indicates the corresponding values we want to insert into those columns.

In [6]:
connection.execute(
    '''
    INSERT INTO person (person_id, name, age)
    VALUES (1, 'Alex', 47);
    '''
)

IntegrityError: UNIQUE constraint failed: person.person_id

Notice how the above error fails with message "UNIQUE constraint failed", because we had already inserted a row with a `person_id` of 1. This is SQLite at work ensuring the integrity of our database.

In [10]:
connection.execute(
    '''
    INSERT INTO person (person_id, name, age)
    VALUES (2, 'Alex', 47);
    '''
)

### Filtering

When we use `SELECT` to pull rows from a table, it is quite likely that we don't want to return every row because often we are looking for something specific:

In [11]:
cursor = connection.execute(
    '''
    SELECT name FROM person
    WHERE age < 40;
    '''
)
print(*cursor.fetchall())
cursor.close()

('Boo',)


Generally the same syntax as before, however we used the `WHERE` keyword to specify a boolean expression that filtered out results we didn't want.

In [12]:
cursor = connection.execute(
    '''
    SELECT name, age FROM person
    WHERE length(name) = 4 AND person_id BETWEEN 1 AND 10;
    '''
)
print(*cursor.fetchall())
cursor.close()

('Alex', 47)


We can also use truthiness as a boolean, where non-zero integers are true, although it is probably best to avoid this:

In [13]:
cursor = connection.execute(
    '''
    SELECT name, age FROM person
    WHERE age;
    '''
)
print(*cursor.fetchall())
cursor.close()

('Boo', 13) ('Alex', 47)


### Ordering

On top of filtering, we can also control the order that the `SELECT` returns data with `ORDER BY`. Note, if we do both, the `WHERE` must come prior to the `SELECT`:

In [14]:
cursor = connection.execute(
    '''
    SELECT name, age FROM person
    ORDER BY age DESC;
    ''' # DESC for descending, ASC for ascending
)
print(*cursor.fetchall())
cursor.close()

('Alex', 47) ('Boo', 13)


### Modifying data

When we're managing a database, it also stands to reason that we'd want to modify existing rows. This can be done with `UPDATE` that requires us to specify `SET`: which is what we want to set, and `WHERE`: which is where we want to set these changes

In [15]:
connection.execute(
    '''
    UPDATE person
    SET age = age + 1
    WHERE person_id = 2
    '''
)

cursor = connection.execute(
    '''
    SELECT name, age FROM person
    '''
)
print(*cursor.fetchall())
cursor.close()

('Boo', 13) ('Alex', 48)


We can also update multiple values as such:

In [16]:
connection.execute(
    '''
    UPDATE person
    SET age = 25, name = 'Bob'
    WHERE person_id = 2;
    '''
)

cursor = connection.execute(
    '''
    SELECT name, age FROM person
    '''
)
print(*cursor.fetchall())
cursor.close()

('Boo', 13) ('Bob', 25)


If we don't specify a `WHERE`, it will simply apply `SET` to every row in the table.

### Deleting things
We can delete rows from tables, as well as tables entirely

In [17]:
connection.execute(
    '''
    DELETE FROM person
    WHERE person_id = 2;
    '''
)

Or to drop an entire table

```SQL
DROP TABLE person;
```

From this point on (just for databases) I'll write examples in markdown instead of executing actual Python just for notes, but these are worth experimenting with.

### NULL
NULL is a value that can be stored that represents the lack of a value. When we fetch data, it translates back to Python as `None` since that seems to make the most sense.

Suppose we have colums a, b, c, all integers and we add the following entry:
```SQL
INSERT into some_table (a, b)
VALUES (1, 2);
```

Fetching this entry form the table would yield the tuple `(1, 2, None)`, since a NULL was stored where we didn't specify a value in c.

We can also use NULL with WHERE to find empty rows in columns:
```SQL
WHERE some_column IS NULL;
```
Notice how we can't do `some_column = NULL`.

When we construct a table, we can also use the keyword `NOT NULL` (similar to `PRIMARY KEY`) to specify that column cannot contain NULL values.

### Relationships

How do we actually apply theoretically building relationships into SQL queries:

```SQL
CREATE TABLE othertable(
    other_id INTEGER NOT NULL PRIMARY KEY
) STRICT;

CREATE TABLE mytable(
    id INTEGER NOT NULL PRIMARY KEY,
    name TEXT,
    related_thing INTEGER NOT NULL,
    FOREIGN KEY (related_thing) REFERENCES othertable(other_id)
) STRICT;
```

Using this syntax we can specify a certain column as a foreign key, which relates to some other row. We can see that this is sort of a one way relationship, so we can build one-to-one and one-to-many quite easily, however for many-to-many we'd likely have to create an intermediary table. (the example in the notes is better than anything I can come up with)

### Joins

Once we establish foreign key relationships, we can use joins to combine the rows of tables. The most common type is the "inner join". Once again, I would say reference the notes because it's explained much thoroughly in a way that I don't think can be summarized adequately.